In [11]:
from __future__ import print_function
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.layers import AveragePooling2D, Input, Flatten, Activation, Dropout, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.regularizers import l2
from keras import backend as K
from keras.models import Sequential
import pandas as pd
import random
import numpy as np
import shutil
from CopyFiles import copyImagesToFiles, createFolder

In [12]:
def sve_jpg(df):
    for item in list(df.new_filename):
        if not '.jpg' in item:
            return False
    return True

In [19]:
#df = pd.read_csv('/home/ivana/repos/Vje-ba/all_data_info.csv')
df = pd.read_csv('/home/vedran/Documents/Fakultet/Strojno učenje/ivana/all_data_info.csv')
seed = 123

print(df.shape)
df.head()

(103250, 12)


,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg


In [20]:
print('Sve su jpg: ' + str(sve_jpg(df)))

Sve su jpg: True


In [21]:
threshold = 300

x = list(df['artist'].value_counts())
# broj umjetnika koji imaju vise ili jednako od 300 slika
print(len([a for a in x if a >= threshold]))
# len(set(x)) #---> ukupan broj umjetnika

57


In [22]:
df3 = df[df['pixelsx'] <= 224]
df3.head()

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
83052,Gene Davis,1968,abstract,213.0,1400.0,54113.0,wikiart,Color Field Painting,Long John,train_and_test,True,5485.jpg
93984,Nakahara Nantenbo,NaN,figurative,220.0,991.0,47267.0,wikiart,Zen,Nantenbo's Hand Print,train_and_test,True,103177.jpg
96579,Nakahara Nantenbo,NaN,NaN,220.0,903.0,47577.0,wikiart,Zen,Crows,train_and_test,True,86849.jpg
97123,Nakahara Nantenbo,NaN,NaN,220.0,887.0,49986.0,wikiart,Zen,Horse,train_and_test,True,67081.jpg
97275,Enji Torei,NaN,calligraphy,200.0,971.0,52459.0,wikiart,Zen,Calligraphy,train_only,True,7504.jpg


In [23]:
df4 = df[df['pixelsy'] <= 224]
df4.head()

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
81744,Jackson Pollock,1948.0,abstract,1427.0,217.0,212782.0,wikiart,Action painting,Summertime: Number 9A,train_and_test,True,54406.jpg
93764,Henry Darger,NaN,illustration,1000.0,220.0,45338.0,wikiart,Art Brut,195 Are unsuccessfully attacked by Glandelinia...,train_and_test,True,94192.jpg
99659,Grant Wood,1939.0,genre painting,800.0,223.0,45784.0,wikiart,Regionalism,A short break from pasture work,train_only,True,65509.jpg
99786,Marianne North,1876.0,marina,944.0,188.0,34087.0,wikiart,Impressionism,"Mouth of the Kuching River, Sarawak, Borneo",train_only,True,89420.jpg
100951,Candido Portinari,1948.0,genre painting,1000.0,170.0,65254.0,wikiart,Expressionism,O Painel Tiradentes,train_and_test,True,83825.jpg


In [6]:
# ćelija za brisanje
from os import path, getcwd
from shutil import copy
from keras.preprocessing import image

moj_direktorij  = getcwd()

print(path.join(path.abspath(path.join(moj_direktorij, '..')), 'images'))
pathh = path.join(path.abspath(path.join(moj_direktorij, '..')), 'images', '1.jpg')
img = image.load_img(pathh)

copy(pathh, '/home/ivana/')
img = image.load_img('/home/ivana/1.jpg')
img.show()

/home/ivana/repos/images


In [24]:
# train, validation, test --- 80, 10, 10
num_train = 240
num_val = 30
num_test = num_val
num_samples = num_train + num_val + num_test
b_size = 60

#lista umjetnika koje ćemo promatrati
temp = df['artist'].value_counts()
artists = temp[temp >= threshold].index.tolist()
# print(artists)

num_artists = len(artists)
print('Prepoznajemo ' + str(num_artists) + ' umjetnika')

train_dfs = []
val_dfs = []
test_dfs = []

for a in artists:
    # PROVJERI KASNIJE ŠTA JE S NA=TRUE
    tmp = df[df['artist'].str.startswith(a)].sample(n=num_samples, random_state=seed)
    # print(tmp.shape)
    t_df = tmp.sample(n=num_train, random_state=seed)
    rest_df = tmp.loc[~tmp.index.isin(t_df.index)] # uzmi komplement od t_df
    # print(rest_df.shape)
    v_df = rest_df.sample(n=num_val, random_state=seed)
    te_df = rest_df.loc[~rest_df.index.isin(v_df.index)]
    
    train_dfs.append(t_df)
    val_dfs.append(v_df)
    test_dfs.append(te_df)
    
    # ovo se pokrene samo jednom!!
    copyImagesToFiles(a, t_df, v_df, te_df)

train_df = pd.concat(train_dfs)
val_df = pd.concat(val_dfs)
test_df = pd.concat(test_dfs)

print('train tablica\t\t', train_df.shape)
print('validation tablica\t', val_df.shape)
print('test tablica\t\t', test_df.shape)

Prepoznajemo 57 umjetnika


NameError: name 'copy' is not defined

In [43]:
def center_crop(img, center_crop_size):
    assert img.shape[2] == 3
    centerw, centerh = img.shape[0] // 2, img.shape[1] // 2
    halfw, halfh = center_crop_size[0] // 2, center_crop_size[1] // 2
    return img[centerw-halfw:centerw+halfw, centerh-halfh:centerh+halfh, :]

# https://jkjung-avt.github.io/keras-image-cropping/
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length, random_cropping=True):
    '''
    Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator
    '''
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            if random_cropping == True:
                batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
            else:
                batch_crops[i] = center_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)


In [44]:
# velicina slika koje dajemo ulaznom sloju mreze
input_shape = (224, 224, 3)
# velicina batch-a
b_size = 30

train_datagen = ImageDataGenerator(
                horizontal_flip=True)

val_datagen = ImageDataGenerator(
                horizontal_flip=True)
test_datagen = ImageDataGenerator()


train_generator = train_datagen.flow_from_directory(
                    '../train',
                    batch_size=b_size,
                    class_mode='categorical')
train_generator = train_datagen.standardize(train_generator)
# na slikama iz train skupa radimo crop na slučajnom mjestu
train_crops = crop_generator(train_generator, 224)

validation_generator = val_datagen.flow_from_directory(
                    '../validation',
                    batch_size=b_size,
                    class_mode='categorical')
validation_generator = val_datagen.standardize(validation_generator)

# na slikama iz validation skupa radimo centralni crop
val_crops = crop_generator(validation_generator, 224, False)


test_generator = test_datagen.flow_from_directory(
                '../test',
                target_size=(224, 224),
                batch_size=b_size)

Found 13680 images belonging to 57 classes.
Found 1710 images belonging to 57 classes.
Found 1710 images belonging to 57 classes.


In [ ]:
i=0
while i < 13680:
    batch_x, batch_y = next(train_crops)
    if batch_x.shape[0]*batch_y.shape[1] > 178956970:
          print i
    i+=1
print('gotov')

Model mreže

In [45]:
# model mreže inspiriran glavnim člankom

model = Sequential()
#nn.Conv2d(3, 32, kernel_size=3 +, stride=2 +, padding=1), # -> 112
model.add(Conv2D(32, kernel_size=3, strides=2, padding="same",input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(32, kernel_size=3, strides=2, padding="same", input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(4*num_artists, input_shape=(6272,))) #nn.Linear(6272, 4*num_artists),
model.add(Activation('relu'))
model.add(Dense(num_artists, input_shape=(4*num_artists,))) #nn.Linear(4*num_artists, num_artists)

# koristimo adamov optimizator i metrika je točnost
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-4),
              metrics=['accuracy'])

# crta tablicu slojeva mreže
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 112, 112, 32)      896       
_________________________________________________________________
activation_25 (Activation)   (None, 112, 112, 32)      0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 112, 112, 32)      128       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_26 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 28, 28, 32)        128       
__________

In [46]:
# treniramo mrežu....

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

model.fit_generator(train_crops,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=3,
                    validation_data=val_crops,
                    validation_steps=STEP_SIZE_VALID,
                    workers=4)

Epoch 1/3
199/456 [============>.................] - ETA: 14:58 - loss: 8.5380 - acc: 0.0256

DecompressionBombError: Image size (669000000 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.

In [ ]:
model.save_weights('prvi_pokusaj300SVE-SLIKE.h5')

In [ ]:
evaluation = model.evaluate_generator(test_generator,
                         steps=test_generator.n//test_generator.batch_size,
                         workers=4,
                         verbose=1)

print(model.metrics_names)
print(evaluation)

In [ ]:
predictions = model.predict_generator(test_generator,
                        steps=test_generator.n//test_generator.batch_size,
                        workers=4,
                        verbose=1)

preds = np.argmax(predictions, axis=1)
print(preds == test_generator.classes)

In [ ]:
# Score trained model.
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])